### Definitions

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import os

db = create_engine(os.environ['GOLEM_CONNSTRING'])

### Locations

In [8]:
locations = pd.read_sql_query('''
    select * from bicyclecounters_locations
''',db).set_index('id')
locations.reset_index().to_csv('csv/locations.csv', index=False)


### Directions

In [9]:
directions = pd.read_sql_query('''
    select * from bicyclecounters_directions
''',db).set_index('id')
directions.reset_index().to_csv('csv/directions.csv', index=False)


### Detections

In [10]:
det = pd.read_sql_query('''
    select 
        date_trunc('day',to_timestamp(measured_from/1000)) as ts,
        locations_id as loc_id,
        directions_id as dir_id,
        sum(value) as value
    from public.bicyclecounters_detections
    group by ts,loc_id,dir_id
''',db)
det['ts'] = pd.to_datetime(det.ts,utc=True)
det.ts = det.ts.dt.tz_convert(tz='Europe/Prague').dt.tz_localize(None)

In [11]:
det['location'] = det.loc_id.map(locations.name)
det['direction'] = det.dir_id.map(directions.name)


In [12]:
det = det.set_index(['location','direction','ts']).value

det = det[~det.index.duplicated()]
det.reset_index().to_csv('csv/detections.csv', index=False)